<a href="https://colab.research.google.com/github/psgrigoletti/santander-bootcamp-2023/blob/main/Desafio1_SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

In [ ]:
!pip install selenium
!pip install yfinance

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import yfinance as yf

# ETL - Extração dos dados
- usando webscrapping, buscar a lista de ativos que compõe o índice Bovespa
- filtrar pelas ações do setor de bancos
- buscar dados do yfinance

# ETL - Transformação dos dados
- pegar valores de minimo, máximo, média e atual
- criar a recomendação baseada nesses valores

# ETL - Load
- gravar um arquivo CSV com as indicações de compra
- dos bancos que foram mais descontados no setor no último ano

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def busca_carteira_teorica(indice, espera=8):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  wd = web_driver()
  wd.get(url)
  wd.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
  sleep(espera)

  wd.find_element(By.LINK_TEXT, "Download").click()
  sleep(espera)

  arquivos = !ls -1t *.csv
  return pd.read_csv(arquivos[0], sep=';', encoding='ISO-8859-1', skipfooter=2, engine="python", thousands='.', decimal=',', header=1, index_col=False)


acoes_ibov = busca_carteira_teorica("IBOV")

In [ ]:
acoes_ibov

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1481593024,2.592,2.592
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734632705,0.702,0.702
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327593725,0.223,2.200
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,995335937,0.601,2.200
4,Bens Indls/Transporte,GOLL4,GOL,PN N2,198184909,0.065,2.200
...,...,...,...,...,...,...,...
81,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,290435910,0.647,9.818
82,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1578931315,0.929,9.818
83,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255230961,0.518,9.818
84,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1095587251,1.655,9.818


# ETL - Transformação

In [ ]:
acoes_ibov['Setor'].unique()

array(['Bens Indls / Máqs e Equips', 'Bens Indls / Mat Transporte',
       'Bens Indls/Transporte', 'Cons N  Básico / Alimentos Processados',
       'Cons N Cíclico / Bebidas', 'Cons N Cíclico / Comércio Distr.',
       'Cons N Cíclico / Pr Pessoal Limp', 'Cons N Ciclico/Agropecuária',
       'Consumo Cíclico / Comércio', 'Consumo Cíclico / Tecid Vest Calç',
       'Consumo Cíclico/Constr Civil', 'Consumo Cíclico/Viagens e Lazer',
       'Diversos', 'Financ e Outros / Explor Imóveis',
       'Financ e Outros / Interms Financs',
       'Financ e Outros / Previd  Seguros',
       'Financeiro e Outros/Serviços Financeiros Diversos',
       'Mats Básicos / Madeira e Papel', 'Mats Básicos / Mineração',
       'Mats Básicos / Químicos', 'Mats Básicos / Sid Metalurgia',
       'Petróleo/ Gás e Biocombustíveis', 'Saúde/Comércio Distr.',
       'Saúde/SM Hosp An.Diag', 'Tec.Informação/Programas Servs',
       'Telecomunicação', 'Utilidade Públ / Água Saneamento',
       'Utilidade Públ / Energ 

In [ ]:
acoes_bancos = acoes_ibov['Código'][acoes_ibov["Setor"] == 'Financ e Outros / Interms Financs']

In [ ]:
codigos_acoes_bancos = [c + ".SA" for c in acoes_bancos]
bancos = pd.DataFrame(columns=["Ticker", "Minimo", "Maximo", "Media", "Atual", "%Minimo", "%Maximo", "%Media"])
for c in codigos_acoes_bancos:
  b = yf.download(c, start="2023-01-01")["Close"]
  atual = b.iloc[0]
  maximo = b.max()
  minimo = b.min()
  media = b.mean()
  perc_min = (minimo-atual)/atual
  perc_max = (maximo-atual)/atual
  perc_media = (media-atual)/atual
  bancos.loc[len(bancos)] = [c, minimo, maximo, media, atual, perc_min, perc_max, perc_media]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
bancos

,Ticker,Minimo,Maximo,Media,Atual,%Minimo,%Maximo,%Media
0,BBDC3.SA,11.240000,15.100000,13.124104,13.190000,-0.147839,0.144807,-0.004996
1,BBDC4.SA,12.570000,17.290001,14.875954,14.750000,-0.147797,0.172203,0.008539
2,BBAS3.SA,32.639999,51.990002,43.389827,33.259998,-0.018641,0.563139,0.304565
3,BPAC11.SA,18.290001,34.080002,25.897442,22.379999,-0.182752,0.522788,0.157169
4,ITSA4.SA,7.780000,9.900000,8.900056,8.360000,-0.069378,0.184211,0.064600
5,ITUB4.SA,23.150000,29.139999,26.470983,24.490000,-0.054716,0.189873,0.080889
6,SANB11.SA,25.520000,31.410000,28.385581,27.629999,-0.076366,0.136808,0.027346


In [ ]:
media_setor = bancos["%Media"].mean()
media_setor

0.09115903466718911

In [ ]:
bancos_escolhidos = bancos[bancos['%Media']<media_setor]
bancos_escolhidos

,Ticker,Minimo,Maximo,Media,Atual,%Minimo,%Maximo,%Media
0,BBDC3.SA,11.24,15.100000,13.124104,13.190000,-0.147839,0.144807,-0.004996
1,BBDC4.SA,12.57,17.290001,14.875954,14.750000,-0.147797,0.172203,0.008539
4,ITSA4.SA,7.78,9.900000,8.900056,8.360000,-0.069378,0.184211,0.064600
5,ITUB4.SA,23.15,29.139999,26.470983,24.490000,-0.054716,0.189873,0.080889
6,SANB11.SA,25.52,31.410000,28.385581,27.629999,-0.076366,0.136808,0.027346


In [ ]:
bancos_escolhidos["Ticker"].to_csv('recomendacoes_setor_bancario.csv', index=False)

In [ ]:
bancos_escolhidos

,Ticker,Minimo,Maximo,Media,Atual,%Minimo,%Maximo,%Media
0,BBDC3.SA,11.24,15.100000,13.124104,13.190000,-0.147839,0.144807,-0.004996
1,BBDC4.SA,12.57,17.290001,14.875954,14.750000,-0.147797,0.172203,0.008539
4,ITSA4.SA,7.78,9.900000,8.900056,8.360000,-0.069378,0.184211,0.064600
5,ITUB4.SA,23.15,29.139999,26.470983,24.490000,-0.054716,0.189873,0.080889
6,SANB11.SA,25.52,31.410000,28.385581,27.629999,-0.076366,0.136808,0.027346
